# Deploy to SageMaker

In [100]:
import sagemaker
from sagemaker import get_execution_role
import boto3
from sagemaker.xgboost.estimator import XGBoost
import boto3
import json

In [101]:
sagemaker_session = sagemaker.Session()

In [102]:
bucket = 'triage-model'
prefix = 'xgboost'
file_name = 'complete_wrangled_data.csv'
output_path = 's3://{}/{}/output'.format(bucket, prefix)
train_input = 's3://{}/data/{}'.format(bucket, file_name)
print("Output path:", output_path)
print("Train path:", train_input)

Output path: s3://triage-model/xgboost/output
Train path: s3://triage-model/data/complete_wrangled_data.csv


In [103]:
role = get_execution_role()
role

'arn:aws:iam::709891711940:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [104]:
estimater = XGBoost(entry_point='scripts/xgb_train.py',
                   framework_version='1.5-1',
                   role=role,
                   instance_count=1,
                   instance_type='ml.m5.2xlarge',
                   output_path=output_path)

In [105]:
estimater.fit({'train': train_input})

2022-09-20 16:47:00 Starting - Starting the training job...
2022-09-20 16:47:24 Starting - Preparing the instances for trainingProfilerReport-1663692419: InProgress
......
2022-09-20 16:48:24 Downloading - Downloading input data...
2022-09-20 16:48:49 Training - Downloading the training image.....[2022-09-20 16:49:38.890 ip-10-2-212-223.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-20:16:49:38:INFO] Imported framework sagemaker_xgboost_container.training
[2022-09-20:16:49:38:INFO] No GPUs detected (normal if no gpus installed)
[2022-09-20:16:49:38:INFO] Invoking user training script.
[2022-09-20:16:49:39:INFO] Module xgb_train does not provide a setup.py. 
Generating setup.py
[2022-09-20:16:49:39:INFO] Generating setup.cfg
[2022-09-20:16:49:39:INFO] Generating MANIFEST.in
[2022-09-20:16:49:39:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  Pre

In [106]:
endpoint_name = 'triage-model'
region = boto3.Session().region_name
sm = boto3.client("sagemaker")
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'a5a8c07c-ea03-479e-a4c6-4bae488a833d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a5a8c07c-ea03-479e-a4c6-4bae488a833d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 20 Sep 2022 16:50:13 GMT'},
  'RetryAttempts': 0}}

In [107]:
endpoint_name = 'triage-model'
predictor = estimater.deploy(endpoint_name=endpoint_name, initial_instance_count=1, instance_type="ml.m5.xlarge")

-----!

# Test Deplyed Model

In [108]:
runtime = boto3.Session().client('sagemaker-runtime')

In [119]:
data_ = {
    'Status': "no longer leaking, wet <1 day",
     'Levels': "1 floor", 
     'Rooms': "5 or more rooms", 
     'Materials': "exterior structure", 
     'Engagement': 'attorney', 
     'source': 'water'
    }

In [120]:
data_

{'Status': 'no longer leaking, wet <1 day',
 'Levels': '1 floor',
 'Rooms': '5 or more rooms',
 'Materials': 'exterior structure',
 'Engagement': 'attorney',
 'source': 'water'}

In [121]:
endpoint_name = 'triage-model'
response = runtime.invoke_endpoint(EndpointName = endpoint_name,# The name of the endpoint we created
                                       ContentType = 'application/json',                 # The data format that is expected
                                       Body = json.dumps(data_))

In [122]:
import json
json.loads(response["Body"].read().decode('utf-8'))

{'service': 'FS'}